In [3]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()
vector = embedder.embed_documents([
    "hi",
    "how",
    "are",
    "you longer sentences because"
])

len(vector)

4

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI() 

cache_dir = LocalFileStore("./.cache/")



splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=500,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader('./files/00713_small.txt', encoding='utf-8')

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,cache_dir
)


vectorstore = Chroma.from_documents(docs, cached_embeddings)



chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)




chain.run("맹견의 범위는 어떻게 되나요?")

Created a chunk of size 807, which is longer than the specified 500
Created a chunk of size 1116, which is longer than the specified 500


'맹견의 범위는 "도사견과 그 잡종의 개, 핏불테리어(아메리칸 핏불테리어를 포함한다)와 그 잡종의 개, 아메리칸 스태퍼드셔 테리어와 그 잡종의 개, 스태퍼드셔 불 테리어와 그 잡종의 개, 로트와일러와 그 잡종의 개"로 정의되어 있습니다.'

In [4]:
results = vectorstore.similarity_search("맹견의 범위는 어떻게 되나요?")

results


[Document(page_content='제2조(맹견의 범위) 「동물보호법」(이하 “법”이라 한다) 제2조제5호가목에 따른 “농림축산식품부령으로 정하는 개”란 다음 각 호를 말한다. 1. 도사견과 그 잡종의 개 2. 핏불테리어(아메리칸 핏불테리어를 포함한다)와 그 잡종의 개 3. 아메리칸 스태퍼드셔 테리어와 그 잡종의 개 4. 스태퍼드셔 불 테리어와 그 잡종의 개 5. 로트와일러와 그 잡종의 개', metadata={'source': './files/00713_small.txt'}),
 Document(page_content='제2조(맹견의 범위) 「동물보호법」(이하 “법”이라 한다) 제2조제5호가목에 따른 “농림축산식품부령으로 정하는 개”란 다음 각 호를 말한다. 1. 도사견과 그 잡종의 개 2. 핏불테리어(아메리칸 핏불테리어를 포함한다)와 그 잡종의 개 3. 아메리칸 스태퍼드셔 테리어와 그 잡종의 개 4. 스태퍼드셔 불 테리어와 그 잡종의 개 5. 로트와일러와 그 잡종의 개', metadata={'source': './files/00713_small.txt'}),
 Document(page_content='제3조(반려동물의 범위) 법 제2조제7호에서 “개, 고양이 등 농림축산식품부령으로 정하는 동물”이란 개, 고양이, 토끼, 페럿, 기니피그 및 햄스터를 말한다.', metadata={'source': './files/00713_small.txt'}),
 Document(page_content='제3조(반려동물의 범위) 법 제2조제7호에서 “개, 고양이 등 농림축산식품부령으로 정하는 동물”이란 개, 고양이, 토끼, 페럿, 기니피그 및 햄스터를 말한다.', metadata={'source': './files/00713_small.txt'})]